<a href="https://colab.research.google.com/github/jaiyue/Movie-Data-Analysis/blob/main/Movie_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Remove the movie dataset to ensure the next download succeeds

In [1]:
!rm -r the-movies-dataset

rm: cannot remove 'the-movies-dataset': No such file or directory


import every tool

In [2]:
!pip install opendatasets
import opendatasets as od
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import ast, json

upload kaggle.json file to download dataset

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hailingcai","key":"3581232c04d8591ebaeedf74bd3de296"}'}

In [4]:
od.download('https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset')

Dataset URL: https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset


100%|██████████| 228M/228M [00:00<00:00, 420MB/s]


In [5]:
!ls /content/the-movies-dataset

credits.csv   links.csv        movies_metadata.csv  ratings_small.csv
keywords.csv  links_small.csv  ratings.csv


In [6]:
movies = pd.read_csv('/content/the-movies-dataset/movies_metadata.csv', low_memory=False)
ratings = pd.read_csv('/content/the-movies-dataset/ratings_small.csv')
credits = pd.read_csv('/content/the-movies-dataset/credits.csv')
keywords = pd.read_csv('/content/the-movies-dataset/keywords.csv')

check data structue and the top 5 row

In [7]:
print(movies.head())
print(movies.info())
print(ratings.head())
print(credits.head())
print(keywords.head())

   adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497         

 Clean dat: delete useless column

In [8]:
movies = movies.drop(['homepage', 'poster_path', 'tagline'], axis=1)

Separate all released movies from other movies

In [9]:
print(movies['status'].unique())

['Released' nan 'Rumored' 'Post Production' 'In Production' 'Planned'
 'Canceled']


In [13]:
released_movies = movies[movies['status'] == 'Released'].copy()
other_movies = movies[movies['status'] != 'Released'].copy()

print("Released movies:", released_movies.shape)
print("Other movies:", other_movies.shape)

Released movies: (45014, 21)
Other movies: (452, 21)


Basic Statistics:
1.  How has the number of movies released each year changed?


*   转换 release_date 为日期格式（无效的设为 NaT）
*   删除无效日期
*   提取 release_year
*   按年份统计电影数量
*   可视化









In [14]:
released_movies['release_date'] = pd.to_datetime(released_movies['release_date'], errors='coerce')
original_count = len(released_movies)
released_movies = released_movies.dropna(subset=['release_date'])
cleaned_count = len(released_movies)
print(f"Removed {original_count - cleaned_count} rows with invalid release_date.")
released_movies['release_year'] = released_movies['release_date'].dt.year
print(released_movies[['title', 'release_year']].head())
movies_per_year = released_movies.groupby('release_year').size()


Removed 78 rows with invalid release_date.
                         title  release_year
0                    Toy Story          1995
1                      Jumanji          1995
2             Grumpier Old Men          1995
3            Waiting to Exhale          1995
4  Father of the Bride Part II          1995


2.  What is the average runtime for movies in different genres?

Financial Analysis

1.   What are the revenue differences between high-budget and low-budget movies?

2.   What objective factors influence movie revenue?

